# Convolutional Networks

### Load data

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow_hub as hub
import os
import PIL.Image as Image
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import sklearn as skl

os.environ["CUDA_VISIBLE_DEVICES"]= "-1" # Force Tensorflow on CPU instead of GPU (seems to avoid an error with my CUDA compatible GPU)

In [ ]:
# Load training data form NPZ file

train_data = "train.npz"
X_tr = np.load(train_data)["features"]
y_tr = np.load(train_data)["labels"]
images_tr = np.load(train_data)["pixels"]
names_tr = np.load(train_data)["names"]

valid_data = "valid.npz"
X_val= np.load(valid_data)["features"]
y_val = np.load(valid_data)["labels"]
images_val = np.load(valid_data)["pixels"]
names_val = np.load(valid_data)["names"]

test_data = "test.npz"
X_te= np.load(test_data)["features"]
y_te = np.load(test_data)["labels"]
images_te = np.load(test_data)["pixels"]
names_te = np.load(test_data)["names"]

# Create a dictionnary for labels
labels_dict = {0: 'bike', 1 : 'car', 2: 'motorcycle', 3: 'other', 4:'truck', 5: 'van'}

### Let's use the train generator to have more images to train on and use in the Convolutional Network.

In [ ]:
# Create image generator
train_generator = ImageDataGenerator(rescale=1/255, rotation_range = 10, horizontal_flip = True, vertical_flip = False, zoom_range = 0.2)
test_generator = ImageDataGenerator(rescale=1/255)

In [ ]:
# Train, validation and test sets


# class_mode = categorical to have a one hot encoded "y" output.

trainset = train_generator.flow_from_directory(
    os.path.join('Course Project - SwissRoads', 'train'), batch_size =500, target_size=(299, 299),
    shuffle=True, color_mode='rgb', class_mode = 'categorical') 

validset = test_generator.flow_from_directory(
    os.path.join('Course Project - SwissRoads', 'valid'),batch_size =500, target_size=(299, 299),
    shuffle=False, color_mode='rgb', class_mode = 'categorical')
testset = test_generator.flow_from_directory(
    os.path.join('Course Project - SwissRoads', 'test'),batch_size =500, target_size=(299, 299),
    shuffle=False,color_mode='rgb', class_mode = 'categorical')

### Let's create a 2 layers convolutional network

In [ ]:
from tensorflow.keras.layers import Dropout

In [ ]:
# Convolutional Network

model = keras.Sequential()


model.add(keras.layers.Conv2D(filters=128, kernel_size=5, strides=2,
                              activation='relu', input_shape=(299, 299, 3)))

model.add(keras.layers.MaxPool2D(pool_size=2))

model.add(keras.layers.Conv2D(filters=64, kernel_size=3, strides=1,
                              activation='relu'))

model.add(keras.layers.MaxPool2D(pool_size=2))


model.add(keras.layers.Flatten())
model.add(Dropout(rate=0.5)) # because of overfitting

model.add(keras.layers.Dense(units=trainset.num_classes, activation='softmax'))


model.summary()

In [ ]:
# Compile the model
model.compile(optimizer=keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['acc'])

In [ ]:
# End training when accuracy stops improving (optional)
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=6)

In [ ]:
trainset.num_classes

In [ ]:
# Train model
history = model.fit_generator(
    generator = trainset, 
    validation_data = validset, 
    epochs=50, 
    callbacks=[early_stopping],
    verbose = 1,
    use_multiprocessing=False
)

### Let's try the same model with numpy arrays and change y with One Hot Encoding to implement in the model

In [ ]:
### Let's use a bigger dataset for training.

images = np.concatenate((images_tr,images_val), axis=0, out=None) 
y = np.concatenate((y_tr,y_val), axis=0, out=None) 

In [ ]:
from sklearn.preprocessing import OneHotEncoder as ohe

nominal_transformer = ohe(handle_unknown='ignore', sparse = False)

y_ohe = nominal_transformer.fit_transform(y.reshape(-1,1))
print(y_ohe.shape)


y_tr_ohe = nominal_transformer.fit_transform(y_tr.reshape(-1,1))
print(y_tr_ohe.shape)

In [ ]:
len(y_ohe[0])

In [ ]:
model2 = keras.Sequential()


model2.add(keras.layers.Conv2D(filters=128, kernel_size=5, strides=2,
                              activation='relu', input_shape=(299, 299, 3)))

model2.add(keras.layers.MaxPool2D(pool_size=2))

model2.add(keras.layers.Conv2D(filters=32, kernel_size=3, strides=1,
                              activation='relu'))

model2.add(keras.layers.MaxPool2D(pool_size=2))


model2.add(keras.layers.Flatten())

model2.add(Dropout(rate=0.5)) # because of overfitting

model2.add(keras.layers.Dense(units = len(y_ohe[0]), activation='softmax'))


model2.summary()

In [ ]:
# Compile the model
model2.compile(optimizer=keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['acc'])

In [ ]:


# Train model
history2 = model2.fit(
    x = images, 
    y = y_ohe, 
    epochs=50,
    validation_split = 0.2,
    callbacks=[early_stopping],
    verbose = 1,
    shuffle = True
)